In [3]:
import cv2
import sys


In [4]:
def get_data(input_path):
    """Parse the data from annotation file

	Args:
		input_path: annotation file path

	Returns:
		all_data: list(filepath, width, height, list(bboxes))
		classes_count: dict{key:class_name, value:count_num}
			e.g. {'Car': 2383, 'Mobile phone': 1108, 'Person': 3745}
		class_mapping: dict{key:class_name, value: idx}
			e.g. {'Car': 0, 'Mobile phone': 1, 'Person': 2}
	"""
    found_bg = False
    all_imgs = {}

    classes_count = {}

    class_mapping = {}

    i = 1

    with open(input_path, 'r') as f:

        print('Parsing annotation files')

        for line in f:

            # Print process
            sys.stdout.write('\r' + 'idx=' + str(i))
            i += 1

            line_split = line.strip().split(',')

            # Make sure the info saved in annotation file matching the format (path_filename, x1, y1, x2, y2, class_name)
            # Note:
            #	One path_filename might has several classes (class_name)
            #	x1, y1, x2, y2 are the pixel value of the origial image, not the ratio value
            #	(x1, y1) top left coordinates; (x2, y2) bottom right coordinates
            #   x1,y1-------------------
            #	|						|
            #	|						|
            #	|						|
            #	|						|
            #	---------------------x2,y2

            (filename, class_name, x1, y1, x2, y2) = line_split

            if class_name not in classes_count:
                classes_count[class_name] = 1
            else:
                classes_count[class_name] += 1

            if class_name not in class_mapping:
                class_mapping[class_name] = len(class_mapping)

            if filename not in all_imgs:
                all_imgs[filename] = {}

                img = cv2.imread('Dataset/' + filename)
                (rows, cols) = img.shape[:2]
                all_imgs[filename]['filepath'] = filename
                all_imgs[filename]['image'] = img
                all_imgs[filename]['width'] = cols
                all_imgs[filename]['height'] = rows
                all_imgs[filename]['bboxes'] = []

            all_imgs[filename]['bboxes'].append({
                                        'class_id': class_mapping[class_name],
                                        'x_center': (int(x1) + int(x2)) / 2.0 / cols,
                                        'y_center': (int(y1) + int(y2)) / 2.0 / rows,
                                        'width': (int(x2) - int(x1)) / cols,
                                        'height': (int(y2) - int(y1)) / rows})

        all_data = []
        for key in all_imgs:
            all_data.append(all_imgs[key])

        return all_data, classes_count, class_mapping


In [5]:
trainImages, classesCount, classMapping= get_data('data.csv')

Parsing annotation files
idx=4888

In [6]:
import yaml

In [7]:
from pathlib import Path

def splitData(numericalDataset,trainName:str,testName:str,trainSize:float=0.8):
    splitIndex = int(trainSize * len(numericalDataset))
    train:list=[]
    test:list=[]
    for i, file in enumerate(numericalDataset):
        if i < splitIndex:
            train.append(file)
            filename= Path(trainName) / Path(file['filepath'])
            cv2.imwrite(filename,file['image'])
        else:
            test.append(file)
            filename= Path(testName) / Path(file['filepath'])
            cv2.imwrite(filename,file['image'])

    return train, test

In [8]:
import os

def create_yolo_labels(dataset, class_mapping, output_dir='labels'):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    for item in dataset:
        filename = item['filepath']
        label_filename = os.path.splitext(filename)[0] + '.txt'
        label_path = os.path.join(output_dir, label_filename)
        
        with open(label_path, 'w') as f:
            for bbox in item['bboxes']:
                line = f"{bbox['class_id']} {bbox['x_center']} {bbox['y_center']} {bbox['width']} {bbox['height']}\n"
                f.write(line)

In [9]:
def create_yolo_yaml(data_paths, class_mapping, output_path='data.yaml'):
    sorted_classes = [class_name for class_name, idx in 
                     sorted(class_mapping.items(), key=lambda x: x[1])]
    
    yaml_data = {
        'path': '.', 
        'train': 'train',
        'val': 'val',
        'test': 'test',
        'nc': len(sorted_classes), 
        'names': sorted_classes  
    }
    
    with open(output_path, 'w') as f:
        yaml.dump(yaml_data, f, default_flow_style=False, sort_keys=False)

    return yaml_data


In [10]:
dataCombined:dict={}
dataCombined['train'], dataCombined['test'] = splitData(trainImages,trainName='train/images',testName='test/images')
dummyTrain,dataCombined['val']=splitData(dataCombined['train'],trainName='train/images',testName='validation/images')
datasetYAMLformat = create_yolo_yaml(dataCombined,classMapping)

In [11]:
create_yolo_labels(dataCombined['train'],classMapping,"train/labels")
create_yolo_labels(dataCombined['test'],classMapping,"test/labels")
create_yolo_labels(dataCombined['val'],classMapping,"val/labels")

In [12]:
from ultralytics import YOLO


In [13]:
secondModel = YOLO("yolo11n.pt")

secondModel.train(
    data="data.yaml",  
    epochs=50,                 
    imgsz=640,                 
    batch=16,                  
    name="asl_yolo11"          
)

New https://pypi.org/project/ultralytics/8.3.235 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.186  Python-3.11.13 torch-2.7.1+cu118 CUDA:0 (NVIDIA GeForce RTX 2060, 6144MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=asl_yolo1110, nbs=64, nms=False, opset=None, optimize=Fals

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x000001BF071506D0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          

In [17]:
secondModel.save(Path("finalModels/secondModel.pt"))

In [18]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pathlib import Path
from ultralytics import YOLO
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report
import warnings
warnings.filterwarnings('ignore')

def generate_test_report(model_path=Path("finalModels/secondModel.pt"), 
                         test_data_path='test/images', 
                         num_samples=6):
    


    model = YOLO(model_path)
    

    test_results = model.val(data='data.yaml', split='test')
    
    test_images = list(Path(test_data_path).glob('*.jpg'))[:num_samples*2]
    
    fig, axes = plt.subplots(2, 3, figsize=(20, 12))
    axes = axes.flatten()
    
    all_predictions = []
    all_confidences = []
    all_ground_truths = []
    
    class_names = {0: 'WBC', 1: 'RBC', 2: 'Platelets'}
    
    for idx, img_path in enumerate(test_images[:num_samples]):
        results = model.predict(source=str(img_path), conf=0.25)
        
        label_path = Path('test/labels') / (img_path.stem + '.txt')
        ground_truth_boxes = []
        
        if label_path.exists():
            with open(label_path, 'r') as f:
                for line in f:
                    parts = line.strip().split()
                    if len(parts) == 5:
                        class_id, x_center, y_center, width, height = map(float, parts)
                        ground_truth_boxes.append({
                            'class_id': int(class_id),
                            'class_name': class_names[int(class_id)],
                            'x_center': x_center,
                            'y_center': y_center,
                            'width': width,
                            'height': height
                        })
        
        img = cv2.imread(str(img_path))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        h, w = img.shape[:2]
        
        for gt_box in ground_truth_boxes:
            x1 = int((gt_box['x_center'] - gt_box['width']/2) * w)
            y1 = int((gt_box['y_center'] - gt_box['height']/2) * h)
            x2 = int((gt_box['x_center'] + gt_box['width']/2) * w)
            y2 = int((gt_box['y_center'] + gt_box['height']/2) * h)
            
            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(img, f"GT: {gt_box['class_name']}", 
                       (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            
            all_ground_truths.append(gt_box['class_id'])
        
        for r in results:
            boxes = r.boxes
            if boxes is not None:
                for box in boxes:
                    x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())
                    conf = float(box.conf[0])
                    cls = int(box.cls[0])
                    
                    cv2.rectangle(img, (x1, y1), (x2, y2), (255, 0, 0), 2)
                    cv2.putText(img, f"{class_names[cls]}: {conf:.2f}", 
                               (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
                    
                    all_predictions.append(cls)
                    all_confidences.append(conf)
        
        axes[idx].imshow(img)
        axes[idx].set_title(f"Test Image: {img_path.stem}")
        axes[idx].axis('off')
    
    for idx in range(num_samples, len(axes)):
        axes[idx].axis('off')
    
    plt.suptitle(f"Sample Predictions on Test Set\n(Green: Ground Truth, Red: Predictions)", 
                 fontsize=16, y=1.02)
    plt.tight_layout()
    plt.savefig('test_predictions_visualization.png', dpi=150, bbox_inches='tight')
    plt.show()

    metrics_data = {
        'Metric': ['mAP50', 'mAP50-95', 'Precision', 'Recall', 'F1-Score'],
        'Value': [
            test_results.box.map50,
            test_results.box.map,
            test_results.box.mp,
            test_results.box.mr,
            2 * (test_results.box.mp * test_results.box.mr) / (test_results.box.mp + test_results.box.mr + 1e-16)
        ]
    }
    
    metrics_df = pd.DataFrame(metrics_data)
    
    class_metrics = []
    for class_id, class_name in class_names.items():
        class_metrics.append({
            'Class': class_name,
            'Precision': test_results.box.mp,  
            'Recall': test_results.box.mr,    
            'mAP50': test_results.box.map50    
        })
    
    class_metrics_df = pd.DataFrame(class_metrics)
    
    if all_predictions and all_ground_truths:

        cm = confusion_matrix(all_ground_truths[:len(all_predictions)], 
                             all_predictions[:len(all_ground_truths)])
        
        plt.figure(figsize=(8, 6))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                    xticklabels=class_names.values(),
                    yticklabels=class_names.values())
        plt.title('Confusion Matrix (Sample)')
        plt.xlabel('Predicted')
        plt.ylabel('Actual')
        plt.tight_layout()
        plt.savefig('confusion_matrix.png', dpi=150)
        plt.show()
    
    print("\n" + "=" * 80)
    print("PERFORMANCE SUMMARY")
    print("=" * 80)
    print(f"Model: {model_path}")
    print(f"Number of test samples analyzed: {len(test_images)}")
    print(f"Overall mAP50: {test_results.box.map50:.4f}")
    print(f"Overall mAP50-95: {test_results.box.map:.4f}")
    print(f"Average Precision: {test_results.box.mp:.4f}")
    print(f"Average Recall: {test_results.box.mr:.4f}")


In [19]:
generate_test_report()

Ultralytics 8.3.186  Python-3.11.13 torch-2.7.1+cu118 CUDA:0 (NVIDIA GeForce RTX 2060, 6144MiB)
YOLO11n summary (fused): 100 layers, 2,582,737 parameters, 0 gradients, 6.3 GFLOPs
val: Fast image access  (ping: 0.10.0 ms, read: 1.60.4 MB/s, size: 39.0 KB)
val: Scanning G:\Job Bullshit\Coding\Blood Cell Detection\test\labels.cache... 73 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 73/73  0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 5/5 0.85it/s 5.9s
                   all         73       1041      0.882      0.961      0.967      0.696
                   WBC         73         74      0.992          1      0.995      0.827
                   RBC         71        908      0.823      0.917      0.943      0.727
             Platelets         28         59      0.832      0.966      0.962      0.535
Speed: 1.7ms preprocess, 6.7ms inference, 0.0ms loss, 7.4ms postprocess per image
Results saved to runs\detect\val6



<Figure size 2000x1200 with 6 Axes>

<Figure size 800x600 with 2 Axes>


PERFORMANCE SUMMARY
Model: finalModels\secondModel.pt
Number of test samples analyzed: 12
Overall mAP50: 0.9667
Overall mAP50-95: 0.6964
Average Precision: 0.8825
Average Recall: 0.9609
